### AVL Trees (Adel'son-Vel'skii and Landis, 1962)

Ideally, we would want our BSTs to be of height **log(n)** where n is the length of the list, but in the worse case where the elements are in sorted order, the height of the BST would be **n** which is not ideal. If we denote the height to be **h**, it can be defined as follows:
- ```h = length of longest path from root to leaf going down```

A tree is ***balanced*** if h is $\theta$log (n) where n is the length of the list. Generally, we can define the height of a node as 
- if node is leaf, ```h(node)=-1```
- ```h(node) = max(h(left_child), h(right_child))+1```

AVL Trees require the following invariant
- For all nodes, |h(left) - h(right)| <=1

N<sub>h</sub> = min. no. of nodes in an AVL tree of height h

N<sub>h</sub> = 1 + N<sub>h-1</sub> + N<sub>h-2</sub>

#### Import classes from Lecture 5

We are going to reuse the classes that we created in Lecture 5, namely BSTAugmented(). This section contains code to use the helper notebook as a package. The code in this section has been taken from [this](https://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Importing%20Notebooks.html) tutorial

In [2]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [9]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [10]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [11]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [12]:
sys.meta_path.append(NotebookFinder())

In [16]:
from helper import BST

 #### Balancing AVL trees
 Given a node x which is the lowest node violating the AVL invariant:
 - if x.right is higher
   1. if x's right child is right-heavy or balanced
       - right-rotate x
   2. if x's right child is left-heavy
       - right-rotate x.right
       - left-rotate x
   

In [17]:
def height(node):
    if node is None:
        return -1
    else:
        return node.height
    

In [18]:
def update_height(node):
    node.height=max(height(node.left), height(node.right))+1

In [34]:
class AVL(BST):
    '''
    Class implementing AVL BST
    Supports insert, find and delete_min in O(lg n)
    Call with non-zero value to see operations list
    '''
    def __init__(self, verbose=0):
        self.root=None
        self.verbose=verbose
        
    def left_rotate(self, x):
        #initial steps. Set y to x.right. Set parents and check for root node
        verbose=self.verbose
        y=x.right
        y.parent=x.parent
        if not y.parent:
            self.root=y
        else:
            if y.parent.left is x:
                y.parent.left=y
            elif y.parent.right is x:
                y.parent.right=y
        #Do the rotation
        x.right=y.left
        if x.right:
            x.right.parent=x
        y.left=x
        x.parent=y
        #update heights of the nodes
        update_height(x)
        update_height(y)
        
    def right_rotate(self, x):
        #initial steps. Set y to x.left. Set parents and check for root node
        verbose=self.verbose
        y=x.left
        y.parent=x.parent
        if not y.parent:
            self.root=y
        else:
            if y.parent.left is x:
                y.parent.left=y
            elif y.parent.right is x:
                y.parent.right=y
        #Do the rotation
        x.left=y.right
        if x.left:
            x.left.parent=x
        y.right=x
        x.parent=y
        #update heights of the nodes
        update_height(x)
        update_height(y)
        
    def insert(self, t):
        verbose=self.verbose
        node = BST.insert(self, t)
        if verbose:
            print(f"Inserted {t}")
            print("Balancing...")
        self.balance(node)
        if verbose:
            print("Balanced")
            
    def balance(self, node):
        verbose=self.verbose
        while node:
            update_height(node)
            if height(node.left)>= 2 + height(node.right):
                if height(node.left.left)>=height(node.left.right):
                    self.right_rotate(node)
                    if verbose:
                        print(f"Right rotated {node.key}")
                else:
                    self.left_rotate(node.left)
                    if verbose:
                        print(f"Left rotated {node.left.key}")
                    self.right_rotate(node)
                    if verbose:
                        print(f"Right rotated {node.key}")
            elif height(node.right)>= 2 + height(node.left):
                if height(node.right.right)>height(node.right.left):
                    self.left_rotate(node)
                    if verbose:
                        print(f"Left rotated {node.key}")
                else:
                    self.right_rotate(node.right)
                    if verbose:
                        print(f"Right rotated {node.right.key}")
                    self.left_rotate(node)
                    if verbose:
                        print(f"Left rotated {node.key}")
            node=node.parent
            
    def delete_min(self):
        verbose=self.verbose
        node,parent=BST.delete_min(self)
        if parent:
            parent_val=parent.key
        else:
            parent_val=None
        if verbose:
            print(f"Deleted {node.key}, child of {parent_val}\nBalancing...")
        self.balance(parent)
        if verbose:
            print("Balanced")

In [37]:
import random
min_range, max_range=-100,100
arr_size=20
A=[random.randint(min_range, max_range) for _ in range(arr_size)]
print(A)
tree_avl=AVL(0)
for item in A:
    tree_avl.insert(item)
    

res=[]
tree_root=tree_avl.root
helper.traverse(tree_root, res)
print(f"Sorted list: {res}")
tree_avl.delete_min()

[-27, -46, -50, 53, -23, 51, 82, 68, 58, 21, -20, 70, -72, -48, 69, -86, -7, -49, 39, -37]
New tree with root value -27
Sorted list: [-86, -72, -50, -49, -48, -46, -37, -27, -23, -20, -7, 21, 39, 51, 53, 58, 68, 69, 70, 82]
